<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/276_MissionOrchestratorAgent_Toolshed_TaskDependency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task dependency resolution utilities

In [ ]:
"""Task dependency resolution utilities"""

from typing import List, Dict, Any, Set, Tuple


def order_tasks_by_dependency(tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Order tasks by their order field (simple sequential ordering).

    Args:
        tasks: List of task dictionaries with 'order' field

    Returns:
        List of tasks sorted by order field
    """
    return sorted(tasks, key=lambda t: t.get("order", 999))


def detect_circular_dependencies(tasks: List[Dict[str, Any]]) -> Tuple[bool, List[str]]:
    """
    Detect circular dependencies in task dependencies.

    Uses DFS to detect cycles in the dependency graph.

    Args:
        tasks: List of task dictionaries with 'task_id' and 'depends_on' fields

    Returns:
        (has_circular_deps, cycle_path): Tuple indicating if cycles exist and path if found
    """
    # Build dependency graph
    graph: Dict[str, List[str]] = {}
    task_ids = set()

    for task in tasks:
        task_id = task.get("task_id")
        if not task_id:
            continue
        task_ids.add(task_id)
        depends_on = task.get("depends_on", [])
        graph[task_id] = depends_on

    # DFS to detect cycles
    visited: Set[str] = set()
    rec_stack: Set[str] = set()
    cycle_path: List[str] = []

    def has_cycle(node: str, path: List[str]) -> bool:
        visited.add(node)
        rec_stack.add(node)
        path.append(node)

        for dep in graph.get(node, []):
            if dep not in task_ids:
                # Dependency doesn't exist - not a cycle, but an error
                continue
            if dep not in visited:
                if has_cycle(dep, path):
                    return True
            elif dep in rec_stack:
                # Found a cycle
                cycle_path.extend(path[path.index(dep):])
                cycle_path.append(dep)
                return True

        rec_stack.remove(node)
        path.pop()
        return False

    # Check all nodes
    for task_id in task_ids:
        if task_id not in visited:
            if has_cycle(task_id, []):
                return (True, cycle_path)

    return (False, [])


def resolve_task_dependencies(tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Resolve task dependencies and return tasks in execution order.

    This function:
    1. Detects circular dependencies
    2. Orders tasks by their dependencies
    3. Returns tasks ready to execute (those with no dependencies or all deps satisfied)

    Args:
        tasks: List of task dictionaries

    Returns:
        List of tasks in execution order (ready to execute first)

    Raises:
        ValueError: If circular dependencies are detected
    """
    # First, check for circular dependencies
    has_cycle, cycle_path = detect_circular_dependencies(tasks)
    if has_cycle:
        raise ValueError(f"Circular dependency detected: {' -> '.join(cycle_path)}")

    # Order tasks by their order field
    ordered_tasks = order_tasks_by_dependency(tasks)

    return ordered_tasks




# ✅ Task Dependency Utilities — Summary

This group of functions teaches your orchestrator **how to understand and safely arrange tasks** before running them.
Think of it as the orchestrator’s **“task traffic controller.”**

The utilities do three important jobs:

---

# ⭐ 1. Ordering Tasks (`order_tasks_by_dependency`)

This is the simplest part:

* Tasks often include an `"order"` field (like 1, 2, 3…)
* This function sorts them from smallest to largest
* If a task has no `"order"`, it gets pushed to the end

➡️ *This gives a basic, predictable sequence to start from.*

---

# ⭐ 2. Detecting Circular Dependencies (`detect_circular_dependencies`)

This is the safety check.

A **circular dependency** happens when tasks depend on each other in a loop:

```
Task A → needs Task B  
Task B → needs Task C  
Task C → needs Task A   ❌
```

That’s impossible to execute—no one can go first.

This function:

* Builds a graph of dependencies
* Uses DFS (depth-first search) to walk through task relationships
* Detects whether the graph loops back on itself
* Returns:

  * `True, [cycle path]` if broken
  * `False, []` if clean

➡️ *This prevents deadlocks and infinite loops.*

---

# ⭐ 3. Resolving Dependencies (`resolve_task_dependencies`)

This is the “boss” function that brings everything together:

1. It calls the circular dependency detector.

   * If a cycle exists → **Stop and throw an error.**

2. It calls the ordering function.

   * Produces a clean, sorted list of tasks.

What it **does not** do (by design):

* It does not try to calculate which tasks are ready to run; that happens later using `get_ready_tasks`.

➡️ *This function creates a “safe, sorted task list” that the orchestrator can use as a foundation.*

---

# 🎯 Why These Utilities Matter

These tools ensure the orchestrator always has:

* A valid task structure
* A safe starting order
* No impossible relationships
* A predictable foundation for scheduling and execution

In short:

> **They keep the orchestrator from crashing or getting stuck.**

They turn messy human-defined workflows into **clean, machine-safe instructions.**

This is one of the pillars of building reliable AI agents.



# Task ordering utilities

In [ ]:
"""Task ordering utilities"""

from typing import List, Dict, Any, Set


def get_ready_tasks(tasks: List[Dict[str, Any]], completed_task_ids: Set[str]) -> List[Dict[str, Any]]:
    """
    Get tasks that are ready to execute (all dependencies satisfied).

    Args:
        tasks: List of all tasks
        completed_task_ids: Set of task_ids that have been completed

    Returns:
        List of tasks ready to execute
    """
    ready = []

    for task in tasks:
        task_id = task.get("task_id")
        depends_on = set(task.get("depends_on", []))

        # Task is ready if all dependencies are completed
        if depends_on.issubset(completed_task_ids):
            ready.append(task)

    return ready




# ✅ Task Ordering Utility — Summary

### **`get_ready_tasks()`**

This function figures out **which tasks are allowed to run right now.**

Every task may have dependencies, like:

* “You must finish Task A before starting Task B.”

This utility checks:

1. **What tasks exist**
2. **Which tasks are already completed**
3. **Which tasks now have *all* their prerequisites done**

If a task’s dependencies are all inside the set of completed tasks, it gets added to the “ready to execute” list.

---

# ⭐ Why It’s Important

This little function enables:

* **Sequential task flow**
* **Dependency-aware execution**
* **Automatic unlocking of future tasks**
* **Dynamic task queue updates**

It basically tells the orchestrator:

> “Here are the tasks you’re allowed to work on next.”

Without it, the agent would either run tasks in the wrong order or get stuck.

---

# 🎯 In one sentence

**`get_ready_tasks()` is the orchestrator’s way of knowing which tasks have all prerequisites cleared and can now safely start.**
